# HELP - Lockman SWIRE Master List Creation

This notebook presents the creation of the HELP master list on the Lockman SWIRE field.

<table>
  <tr>
    <th>Survey/Telescope/Instrument</th>
    <th>Filters</th>
    <th>Location</th>
    <th>Notes</th>
  </tr>
  <tr>
    <td>UKIDSS DXS/UKIRT/WFCAM/</td>
    <td>J,K</td>
    <td>dmu0_UKIDSS-DXS</td>
    <td></td>
  </tr>
  <tr>
    <td>SWIRE/Spitzer/IRAC MIPS</td>
    <td>IRAC1234 & MIPS123</td>
    <td>dmu0_DataFusion-Spitzer</td>
    <td></td>
  </tr>
  <tr>
    <td>SERVS/Spitzer/IRAC		</td>
    <td>IRAC12</td>
    <td>dmu0_DataFusion-Spitzer</td>
    <td></td>
  </tr>
  <tr>
    <td>PS1 3PSS/Pan-STARRS1</td>
    <td>grizy</td>
    <td>dmu0_PanSTARRS1-3SS</td>
    <td></td>
  </tr>
  <tr>
    <td>PS1 MDS/Pan-STARRS1</td>
    <td>grizy</td>
    <td></td>
    <td>Awaiting release...</td>
  </tr>
  <tr>
    <td>SpARCS/CFHT/MegaCam</td>
    <td></td>
    <td>dmu0_SpARCS</td>
    <td></td>
  </tr>
  <tr>
    <td>INT/WFC				</td>
    <td>u,g,r,i,z</td>
    <td>dmu0_INTWFC</td>
    <td></td>
  </tr>
</table>

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
7452f47 (Thu Mar 23 14:08:22 2017 +0000)


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from astropy.table import Column, Table
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np
import seaborn as sns
from pymoc import MOC
from matplotlib_venn import venn3

from herschelhelp_internal import flagging, utils, masterlist

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
import locale
locale.setlocale(locale.LC_ALL, 'en_GB')

'en_GB'

In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## I - Pristine catalogues preparation

### I.a - Isaac Newton Telescope / Wide Field Camera (INT/WFC)

Isaac Newton Telescope / Wide Field Camera (INT/WFC) catalogue: the catalogue comes from `dmu0_INTWFC`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The stellarity;
- The magnitude for each band in aperture 4 ($1.2 * \sqrt{2}$ arcsec = 1.7 arcsec).
    TODO: Replace this with the 3 arcsec one under recomendation of Mattia.
- The kron magnitude to be used as total magnitude (no “auto” magnitude is provided).

We don't know when the maps have been observed. We will use the year of the reference paper.

In [ ]:
wfc = Table.read("../../dmu0/dmu0_INTWFC/data/lh_intwfc_v2.1_HELP_coverage.fits")[
    'id', 'ra', 'decl', 'pstar',     
    'umag4', 'uemag4', 'ukronmag', 'uekronmag',
    'gmag4', 'gemag4', 'gkronmag', 'gekronmag',
    'rmag4', 'remag4', 'rkronmag', 'rekronmag',
    'imag4', 'iemag4', 'ikronmag', 'iekronmag',
    'zmag4', 'zemag4', 'zkronmag', 'zekronmag']
wfc = Table(wfc.as_array(), names=[
    'wfc_id', 'wfc_ra', 'wfc_dec', 'wfc_stellarity',
    'm_app_wfc_u', 'merr_app_wfc_u', 'm_wfc_u', 'merr_wfc_u',
    'm_app_wfc_g', 'merr_app_wfc_g', 'm_wfc_g', 'merr_wfc_g',
    'm_app_wfc_r', 'merr_app_wfc_r', 'm_wfc_r', 'merr_wfc_r',
    'm_app_wfc_i', 'merr_app_wfc_i', 'm_wfc_i', 'merr_wfc_i',
    'm_app_wfc_z', 'merr_app_wfc_z', 'm_wfc_z', 'merr_wfc_z',
])
wfc_epoch = 2011
wfc_moc = MOC(filename="../../dmu0/dmu0_INTWFC/data/lh_intwfc_v2.1_HELP_coverage_MOC.fits")

# Adding flux and band-flag columns
for col in wfc.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        flux, error = utils.mag_to_flux(np.array(wfc[col]), np.array(wfc[errcol]))

        # Fluxes are added in µJy
        wfc.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        wfc.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        wfc.add_column(Column(np.zeros(len(wfc), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
wfc[:10].show_in_notebook()

### I.b - UKIRT Infrared Deep Sky Survey / Deep Extragalactic Survey (UKIDSS/DXS)

The catalogue was provided by Eduardo from an extraction made by Mattia. The catalogue is quite different from what can be extracted from the UKIDSS database (FIXME: Mattia to explain why). The descriptions of the columns of the DXS table in the UKIDSS database is available there: http://horus.roe.ac.uk/wsa/www/WSA_TABLE_dxsSourceSchema.html. It is stored in `dmu0_UKIDSS-DXS`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The stellarity;
- The magnitude for each band in apertude 3 (2 arcsec).
- The kron magnitude to be used as total magnitude (no “auto” magnitude is provided).

The catalogue has been cross-matched with 2MASS catalogue to check that the magnitudes are AB ones.

A query to the UKIDSS database with 242.9+55.071 position returns a list of images taken between 2007 and 2009. Let's take 2008 for the epoch.


In [ ]:
dxs = Table.read("../../dmu0/dmu0_ELAIS-N1_DXS/data/en1_ukidss_dr10plus_merged_HELP_coverage.fits")[
    'sourceid', 'RA', 'Dec', 'PSTAR',     
    'JAPERMAG3', 'JAPERMAG3ERR', 'JKRONMAG', 'JKRONMAGERR',
    'HAPERMAG3', 'HAPERMAG3ERR', 'HKRONMAG', 'HKRONMAGERR',
    'KAPERMAG3', 'KAPERMAG3ERR', 'KKRONMAG', 'KKRONMAGERR']
dxs = Table(dxs.as_array(), names=[
    'dxs_id', 'dxs_ra', 'dxs_dec', 'dxs_stellarity',
    'm_app_ukidss_j', 'merr_app_ukidss_j', 'm_ukidss_j', 'merr_ukidss_j',
    'm_app_ukidss_h', 'merr_app_ukidss_h', 'm_ukidss_h', 'merr_ukidss_h',
    'm_app_ukidss_k', 'merr_app_ukidss_k', 'm_ukidss_k', 'merr_ukidss_k'
])
dxs_epoch = 2008
dxs_moc = MOC(filename="../../dmu0/dmu0_ELAIS-N1_DXS/data/en1_ukidss_dr10plus_merged_HELP_coverage_MOC.fits")

# Adding flux and band-flag columns
for col in dxs.colnames:
    if col.startswith('m_'):
        errcol = "merr{}".format(col[1:])
        
        # DXS uses a huge negative number for missing values
        dxs[col][dxs[col] < -100] = np.nan
        dxs[errcol][dxs[errcol] < -100] = np.nan
        
        flux, error = utils.mag_to_flux(np.array(dxs[col]), np.array(dxs[errcol]))

        # Fluxes are added in µJy
        dxs.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        dxs.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        dxs.add_column(Column(np.zeros(len(dxs), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.        
        

In [ ]:
dxs[:10].show_in_notebook()

### I.c - Spitzer datafusion SERVS

The Spitzer catalogues were produced by the datafusion team are available in the HELP virtual observatory server. They are described there: http://vohedamtest.lam.fr/browse/df_spitzer/q.

Lucia told Yannick that the magnitudes are aperture corrected.

In the catalouge, we keep:

- The internal identifier (this one is only in HeDaM data);
- The position;
- The fluxes in aperture 2 (1.9 arcsec);
- The “auto” flux (which seems to be the Kron flux);
- The stellarity in each band

A query of the position in the Spitzer heritage archive show that the SERVS-ELAIS-N1 images were observed in 2009. Let's take this as epoch.

In [ ]:
servs = Table.read("../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SERVS_ELAIS-N1.fits")[
    'internal_id', 'ra_12', 'dec_12',   
    'flux_aper_2_1', 'fluxerr_aper_2_1', 'flux_auto_1', 'fluxerr_auto_1', 'class_star_1',
    'flux_aper_2_2', 'fluxerr_aper_2_2', 'flux_auto_2', 'fluxerr_auto_2', 'class_star_2']

servs = Table(servs.as_array(), names=[
    'servs_intid', 'servs_ra', 'servs_dec',
    'f_app_servs_irac1', 'ferr_app_servs_irac1', 'f_servs_irac1', 'ferr_servs_irac1', 'servs_stellarity_irac1',
    'f_app_servs_irac2', 'ferr_app_servs_irac2', 'f_servs_irac2', 'ferr_servs_irac2', 'servs_stellarity_irac2'
])
servs_epoch = 2009
servs_moc = MOC(filename="../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SERVS_ELAIS-N1_MOC.fits")

# Adding magnitude and band-flag columns
for col in servs.colnames:
    if col.startswith('f_'):
        errcol = "ferr{}".format(col[1:])
        
        magnitude, error = utils.flux_to_mag(
            np.array(servs[col])/1.e6, np.array(servs[errcol])/1.e6)
        # Note that some fluxes are 0.
        
        servs.add_column(Column(magnitude, name="m{}".format(col[1:])))
        servs.add_column(Column(error, name="m{}".format(errcol[1:])))
        
        # Band-flag column
        servs.add_column(Column(np.zeros(len(servs), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
servs[:10].show_in_notebook()

### I.d - Spitzer datafusion SWIRE

The Spitzer catalogues were produced by the datafusion team are available in the HELP virtual observatory server. They are described there: http://vohedamtest.lam.fr/browse/df_spitzer/q.

Lucia told that the magnitudes are aperture corrected.

In the catalouge, we keep:

We keep:
- The internal identifier (this one is only in HeDaM data);
- The position;
- The fluxes in aperture 2 (1.9 arcsec) for IRAC bands.
- The Kron flux;
- The stellarity in each band

A query of the position in the Spitzer heritage archive show that the ELAIS-N1 images were observed in 2004. Let's take this as epoch.

We do not use the MIPS fluxes as they will be extracted on MIPS maps using XID+.

In [ ]:
swire = Table.read("../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SWIRE_Lockman-SWIRE.fits")[
    'internal_id', 'ra_spitzer', 'dec_spitzer',      
    'flux_ap2_36', 'uncf_ap2_36', 'flux_kr_36', 'uncf_kr_36', 'stell_36',
    'flux_ap2_45', 'uncf_ap2_45', 'flux_kr_45', 'uncf_kr_45', 'stell_45',
    'flux_ap2_58', 'uncf_ap2_58', 'flux_kr_58', 'uncf_kr_58', 'stell_58',
    'flux_ap2_80', 'uncf_ap2_80', 'flux_kr_80', 'uncf_kr_80', 'stell_80']

swire = Table(swire.as_array(), names=[
    'swire_intid', 'swire_ra', 'swire_dec',
    'f_app_swire_irac1', 'ferr_app_swire_irac1', 'f_swire_irac1', 'ferr_swire_irac1', 'swire_stellarity_irac1',
    'f_app_swire_irac2', 'ferr_app_swire_irac2', 'f_swire_irac2', 'ferr_swire_irac2', 'swire_stellarity_irac2',
    'f_app_swire_irac3', 'ferr_app_swire_irac3', 'f_swire_irac3', 'ferr_swire_irac3', 'swire_stellarity_irac3',
    'f_app_swire_irac4', 'ferr_app_swire_irac4', 'f_swire_irac4', 'ferr_swire_irac4', 'swire_stellarity_irac4'
])
swire_epoch = 2004
swire_moc = MOC(filename="../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SWIRE_Lockman-SWIRE_MOC.fits")
        
# Adding magnitude and band-flag columns
for col in swire.colnames:
    if col.startswith('f_'):
        errcol = "ferr{}".format(col[1:])
        
        magnitude, error = utils.flux_to_mag(
            np.array(swire[col])/1.e6, np.array(swire[errcol])/1.e6)
        # Note that some fluxes are 0.
        
        swire.add_column(Column(magnitude, name="m{}".format(col[1:])))
        swire.add_column(Column(error, name="m{}".format(errcol[1:])))
        
        # Band-flag column
        swire.add_column(Column(
                np.zeros(len(swire), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
swire[:10].show_in_notebook()

### I.e -  Spitzer Adaptation of the Red-sequence Cluster Survey (SpARCS)

This catalogue comes from `dmu0_SpARCS`. Alexandru Tudorica confirmed that the magnitudes are AB ones and are not aperture corrected.

In the catalogue, we keep:

- The internal identifier (this one is only in HeDaM data);
- The position;
- The ugrz magnitudes in the 8th aperture (11×0.186=2.046 arcsec).
- The "auto" magnitudes.

Note that there are y band columns because we combined all the SpARCS data in HeDaM, but there is no y data for the ELAIS-N1 sources.

The maps on the web page indicate they were observed in 2012 (or late 2011). Let's use 2012 as epoch.

There is a second notebook `sparcs_aperture_correction.ipynb` detailing how we choose some values for the aperture correction.

In [ ]:
AP_INDEX = 7  # Index of the aperture to keep (2.046 arcsec).

# Index of the target aperture when doing aperture correction
# (see 'sparcs_aperture_correction' notebook).
AP_TARG_INDEX = {
    'u': 15,
    'g': 15,
    'r': 15,
    'z': 15
}

# Magnitude range for aperture correction (see 'sparcs_aperture_correction' notebook).
APCOR_MAG_LIMITS = {
    'u': (17., 22.),
    'g': (17., 23.),
    'r': (16., 23.),
    'z': (16., 22.)
}

sparcs_epoch = 2012
sparcs_moc = MOC(filename="../../dmu0/dmu0_SpARCS/data/SpARCS_HELP_Lockman-SWIRE_MOC.fits")

sparcs_tmp = Table.read("../../dmu0/dmu0_SpARCS/data/SpARCS_HELP_Lockman-SWIRE.fits")[
    'internal_id', 'ALPHA_J2000', 'DELTA_J2000', 'CLASS_STAR',      
    'MAG_APER_r', 'MAGERR_APER_r', 'MAG_AUTO_r', 'MAGERR_AUTO_r',
    'MAG_APER_u', 'MAGERR_APER_u', 'MAG_AUTO_u', 'MAGERR_AUTO_u',
    'MAG_APER_g', 'MAGERR_APER_g', 'MAG_AUTO_g', 'MAGERR_AUTO_g',
    'MAG_APER_z', 'MAGERR_APER_z', 'MAG_AUTO_z', 'MAGERR_AUTO_z']

sparcs = Table(
    data = sparcs_tmp['internal_id', 'ALPHA_J2000', 'DELTA_J2000', 'CLASS_STAR'],
    names = ['sparcs_intid', 'sparcs_ra', 'sparcs_dec', 'sparcs_stellarity'])

for band in ['u', 'g', 'r', 'z']:
    
    # Aperture magnitudes
    mag_aper = sparcs_tmp["MAG_APER_{}".format(band)][:, AP_INDEX]
    mag_aper_target = sparcs_tmp["MAG_APER_{}".format(band)][:, AP_TARG_INDEX[band]]
    magerr_aper = sparcs_tmp["MAGERR_APER_{}".format(band)][:, AP_INDEX]
    
    # Set bad values (99.0) to NaN
    mask = (mag_aper > 90) | (mag_aper_target > 90) | (magerr_aper > 90)
    mag_aper[mask] = np.nan
    mag_aper_target[mask] = np.nan
    magerr_aper[mask] = np.nan
    
    # Aperture correction
    mag_diff, num, std = utils.aperture_correction(
        mag_aper, mag_aper_target, sparcs['sparcs_stellarity'],
        mag_min=APCOR_MAG_LIMITS[band][0], mag_max=APCOR_MAG_LIMITS[band][1]
        )
    print("Aperture correction for SpARCS band {}:".format(band))
    print("Difference: {}".format(mag_diff))
    print("Number of source used: {}".format(num))
    print("RMS: {}".format(std))
    print("")
    mag_aper += mag_diff
    
    sparcs.add_column(Column(
        data = mag_aper.data,
        name = "m_app_cfht_megacam_{}".format(band)
    ))
    sparcs.add_column(Column(
        data = magerr_aper.data,
        name = "merr_app_cfht_megacam_{}".format(band)
    ))
    
    # Computing the aperture flux columns
    flux_aper, fluxerr_aper = utils.mag_to_flux(mag_aper.data, magerr_aper.data)
    
    sparcs.add_column(Column(
        data = flux_aper * 1.e6,
        name = "f_app_cfht_megacam_{}".format(band)
    ))
    sparcs.add_column(Column(
        data = fluxerr_aper * 1.e6,
        name = "ferr_app_cfht_megacam_{}".format(band)
    ))
    
    # Auto magnitudes
    mag_auto = sparcs_tmp["MAG_AUTO_{}".format(band)]
    magerr_auto = sparcs_tmp["MAGERR_AUTO_{}".format(band)]
    
    # Set bad values (99.0) to NaN
    mask = (mag_auto > 90) | (magerr_auto > 90)
    mag_auto[mask] = np.nan
    magerr_auto[mask] = np.nan    
    
    sparcs.add_column(Column(
        data = mag_auto,
        name = "m_cfht_megacam_{}".format(band)
    ))
    sparcs.add_column(Column(
        data = magerr_auto,
        name = "merr_cfht_megacam_{}".format(band)
    ))
    
    # Computing the flux columns
    flux, fluxerr = utils.mag_to_flux(mag_auto, magerr_auto)
    
    sparcs.add_column(Column(
        data = flux * 1.e6,
        name = "f_cfht_megacam_{}".format(band)
    ))
    sparcs.add_column(Column(
        data = fluxerr * 1.e6,
        name = "ferr_cfht_megacam_{}".format(band)
    ))
    
    # Band-flag column
    sparcs.add_column(Column(np.zeros(len(sparcs), dtype=bool), 
                             name="flag_cfht_megacam_{}".format(band)
    ))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
sparcs[:10].show_in_notebook()

### I.e -  Pan-STARRS1 telescope PS1 3PSS survey

This catalogue comes from `dmu0_PanSTARRS1-3SS`. 

In the catalogue, we keep: ?????

- The internal identifier (this one is only in HeDaM data);
- The position;
- The ugrz magnitudes in the 8th aperture (11×0.186=2.046 arcsec).
- The "auto" magnitudes.

Epoch?



In [ ]:
#TODO - Everything!
panstarrs = Table.read("../../dmu0/dmu0_PanSTARRS1-3SS/data/PanSTARRS1-3SS_Lockman-SWIRE.fits")[
    'internal_id', 'ra_12', 'dec_12',   
    'flux_aper_2_1', 'fluxerr_aper_2_1', 'flux_auto_1', 'fluxerr_auto_1', 'class_star_1',
    'flux_aper_2_2', 'fluxerr_aper_2_2', 'flux_auto_2', 'fluxerr_auto_2', 'class_star_2']

panstarrs = Table(servs.as_array(), names=[
    'servs_intid', 'servs_ra', 'servs_dec',
    'f_app_servs_irac1', 'ferr_app_servs_irac1', 'f_servs_irac1', 'ferr_servs_irac1', 'servs_stellarity_irac1',
    'f_app_servs_irac2', 'ferr_app_servs_irac2', 'f_servs_irac2', 'ferr_servs_irac2', 'servs_stellarity_irac2'
])
panstarrs_epoch = 2009 
panstarrs_moc = MOC(filename="../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SERVS_ELAIS-N1_MOC.fits")

# Adding magnitude and band-flag columns
for col in panstarrs.colnames:
    if col.startswith('f_'):
        errcol = "ferr{}".format(col[1:])
        
        magnitude, error = utils.flux_to_mag(
            np.array(servs[col])/1.e6, np.array(servs[errcol])/1.e6)
        # Note that some fluxes are 0.
        
        servs.add_column(Column(magnitude, name="m{}".format(col[1:])))
        servs.add_column(Column(error, name="m{}".format(errcol[1:])))
        
        # Band-flag column
        servs.add_column(Column(np.zeros(len(servs), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
panstarrs[:10].show_in_notebook()

## II - Removal of duplicated sources


In [ ]:
nb_wfc_orig = len(wfc)
wfc = masterlist.remove_duplicates(
    wfc, 'wfc_ra', 'wfc_dec', 
    sort_col=['merr_app_wfc_r', 'merr_app_wfc_u', 'merr_app_wfc_g', 'merr_app_wfc_z'],
    flag_name='wfc_flag_cleaned')
nb_wfc = len(wfc)
print("WFC initial catalogue as {} sources.".format(nb_wfc_orig))
print("WFC cleaned catalogue as {} sources ({} removed).".format(nb_wfc, nb_wfc_orig - nb_wfc))
print("WFC has {} sources flagged as having been cleaned".format(np.sum(wfc['wfc_flag_cleaned'])))

In [ ]:
nb_dxs_orig = len(dxs)
dxs = masterlist.remove_duplicates(
    dxs, 'dxs_ra', 'dxs_dec', 
    sort_col=['merr_app_ukidss_j', 'merr_app_ukidss_h', 'merr_app_ukidss_k'],
    flag_name='dxs_flag_cleaned')
nb_dxs = len(dxs)
print("DXS initial catalogue as {} sources.".format(nb_dxs_orig))
print("DXS cleaned catalogue as {} sources ({} removed).".format(nb_dxs, nb_dxs_orig - nb_dxs))
print("DXS has {} sources flagged as having been cleaned".format(np.sum(dxs['dxs_flag_cleaned'])))

In [ ]:
nb_servs_orig = len(servs)
servs = masterlist.remove_duplicates(
    servs, 'servs_ra', 'servs_dec', 
    sort_col=['ferr_app_servs_irac1', 'ferr_app_servs_irac2'],
    flag_name='servs_flag_cleaned')
nb_servs = len(servs)
print("SERVS initial catalogue as {} sources.".format(nb_servs_orig))
print("SERVS cleaned catalogue as {} sources ({} removed).".format(nb_servs, nb_servs_orig - nb_servs))
print("SERVS has {} sources flagged as having been cleaned".format(np.sum(servs['servs_flag_cleaned'])))

In [ ]:
nb_swire_orig = len(swire)
swire = masterlist.remove_duplicates(
    swire, 'swire_ra', 'swire_dec', 
    sort_col=['ferr_app_swire_irac1', 'ferr_app_swire_irac2', 
              'ferr_app_swire_irac3', 'ferr_app_swire_irac4'],
    flag_name='swire_flag_cleaned')
nb_swire = len(swire)
print("SWIRE initial catalogue as {} sources.".format(nb_swire_orig))
print("SWIRE cleaned catalogue as {} sources ({} removed).".format(nb_swire, nb_swire_orig - nb_swire))
print("SWIRE has {} sources flagged as having been cleaned".format(np.sum(swire['swire_flag_cleaned'])))

In [ ]:
nb_sparcs_orig = len(sparcs)
sparcs = masterlist.remove_duplicates(
    sparcs, 'sparcs_ra', 'sparcs_dec', 
    sort_col=['merr_app_cfht_megacam_r', 'merr_app_cfht_megacam_u',
              'merr_app_cfht_megacam_g', 'merr_app_cfht_megacam_z'],
    flag_name='sparcs_flag_cleaned')
nb_sparcs = len(sparcs)
print("SpARCS initial catalogue as {} sources.".format(nb_sparcs_orig))
print("SpARCS cleaned catalogue as {} sources ({} removed).".format(nb_sparcs, nb_sparcs_orig - nb_sparcs))
print("SpARCS has {} sources flagged as having been cleaned".format(np.sum(sparcs['sparcs_flag_cleaned'])))

In [ ]:
nb_panstarrs_orig = len(panstarrs)
panstarrs = masterlist.remove_duplicates(
    panstarrs, 'panstarrs_ra', 'panstarrs_dec', 
    sort_col=['merr_app_panstarrs_g', 
              'merr_app_panstarrs_r',
              'merr_app_panstarrs_i', 
              'merr_app_panstarrs_z', 
              'merr_app_panstarrs_y'],
    flag_name='panstarrs_flag_cleaned')
nb_panstarrs = len(panstarrs)
print("PanSTARRS initial catalogue as {} sources.".format(nb_panstarrs_orig))
print("PanSTARRS cleaned catalogue as {} sources ({} removed).".format(nb_panstarrs, nb_panstarrs_orig - nb_panstarrs))
print("PanSTARRS has {} sources flagged as having been cleaned".format(np.sum(panstarrs['panstarrs_flag_cleaned'])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_ELAIS-N1.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
masterlist.nb_astcor_diag_plot(
    wfc['wfc_ra'], wfc['wfc_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
wfc_delta_ra, wfc_delta_dec = utils.astrometric_correction(
    SkyCoord(wfc['wfc_ra'], wfc['wfc_dec']),
    gaia_coords
)
wfc['wfc_ra'] += wfc_delta_ra.to(u.deg)
wfc['wfc_dec'] += wfc_delta_dec.to(u.deg)

print("WFC delta RA / delta Dec: {} / {}".format(wfc_delta_ra, wfc_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    dxs['dxs_ra'], dxs['dxs_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
dxs_delta_ra, dxs_delta_dec = utils.astrometric_correction(
    SkyCoord(dxs['dxs_ra'], dxs['dxs_dec']),
    gaia_coords
)
dxs['dxs_ra'] += dxs_delta_ra.to(u.deg)
dxs['dxs_dec'] += dxs_delta_dec.to(u.deg)

print("DXS delta RA / delta Dec: {} / {}".format(dxs_delta_ra, dxs_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    servs['servs_ra'], servs['servs_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
servs_delta_ra, servs_delta_dec = utils.astrometric_correction(
    SkyCoord(servs['servs_ra'], servs['servs_dec']),
    gaia_coords
)
servs['servs_ra'] += servs_delta_ra.to(u.deg)
servs['servs_dec'] += servs_delta_dec.to(u.deg)

print("SERVS delta RA / delta Dec: {} / {}".format(servs_delta_ra, servs_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    swire['swire_ra'], swire['swire_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
swire_delta_ra, swire_delta_dec = utils.astrometric_correction(
    SkyCoord(swire['swire_ra'], swire['swire_dec']),
    gaia_coords
)
swire['swire_ra'] += swire_delta_ra.to(u.deg)
swire['swire_dec'] += swire_delta_dec.to(u.deg)

print("SWIRE delta RA / delta Dec: {} / {}".format(swire_delta_ra, swire_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    sparcs['sparcs_ra'], sparcs['sparcs_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
sparcs_delta_ra, sparcs_delta_dec = utils.astrometric_correction(
    SkyCoord(sparcs['sparcs_ra'], sparcs['sparcs_dec']),
    gaia_coords
)
sparcs['sparcs_ra'] += sparcs_delta_ra.to(u.deg)
sparcs['sparcs_dec'] += sparcs_delta_dec.to(u.deg)

print("SpARCS delta RA / delta Dec: {} / {}".format(sparcs_delta_ra, sparcs_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    panstarrs['panstarrs_ra'], panstarrs['panstarrs_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
panstarrs_delta_ra, panstarrs_delta_dec = utils.astrometric_correction(
    SkyCoord(panstarrs['panstarrs_ra'], panstarrs['panstarrs_dec']),
    gaia_coords
)
panstarrs['panstarrs_ra'] += panstarrs_delta_ra.to(u.deg)
panstarrs['panstarrs_dec'] += panstarrs_delta_dec.to(u.deg)

print("PanSTARRS delta RA / delta Dec: {} / {}".format(sparcs_delta_ra, sparcs_delta_dec))

## IV - Flagging Gaia objects

In [ ]:
wfc.add_column(
    flagging.gaia_flag_column(
        SkyCoord(wfc['wfc_ra'], wfc['wfc_dec']),
        wfc_epoch,
        gaia
    )
)
wfc['flag_gaia'].name = 'wfc_flag_gaia'
print("{} sources flagged.".format(np.sum(wfc['wfc_flag_gaia'] > 0)))

In [ ]:
dxs.add_column(
    flagging.gaia_flag_column(
        SkyCoord(dxs['dxs_ra'], dxs['dxs_dec']),
        dxs_epoch,
        gaia
    )
)
dxs['flag_gaia'].name = 'dxs_flag_gaia'
print("{} sources flagged.".format(np.sum(dxs['dxs_flag_gaia'] > 0)))

In [ ]:
servs.add_column(
    flagging.gaia_flag_column(
        SkyCoord(servs['servs_ra'], servs['servs_dec']),
        servs_epoch,
        gaia
    )
)
servs['flag_gaia'].name = 'servs_flag_gaia'
print("{} sources flagged.".format(np.sum(servs['servs_flag_gaia'] > 0)))

In [ ]:
swire.add_column(
    flagging.gaia_flag_column(
        SkyCoord(swire['swire_ra'], swire['swire_dec']),
        swire_epoch,
        gaia
    )
)
swire['flag_gaia'].name = 'swire_flag_gaia'
print("{} sources flagged.".format(np.sum(swire['swire_flag_gaia'] > 0)))

In [ ]:
sparcs.add_column(
    flagging.gaia_flag_column(
        SkyCoord(sparcs['sparcs_ra'], sparcs['sparcs_dec']),
        sparcs_epoch,
        gaia
    )
)
sparcs['flag_gaia'].name = 'sparcs_flag_gaia'
print("{} sources flagged.".format(np.sum(sparcs['sparcs_flag_gaia'] > 0)))

In [ ]:
panstarrs.add_column(
    flagging.gaia_flag_column(
        SkyCoord(panstarrs['panstarrs_ra'], panstarrs['panstarrs_dec']),
        panstarrs_epoch,
        gaia
    )
)
panstarrs['flag_gaia'].name = 'panstarrs_flag_gaia'
print("{} sources flagged.".format(np.sum(panstarrs['panstarrs_flag_gaia'] > 0)))

## IV - Flagging objects near bright stars


## V- Merging the catalogues

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(wfc['wfc_ra'], wfc['wfc_dec']),
    SkyCoord(dxs['dxs_ra'], dxs['dxs_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
wfc['wfc_ra'].name = 'ra'
wfc['wfc_dec'].name = 'dec'
masterlist_catalogue = masterlist.merge_catalogues(
    wfc, dxs, "dxs_ra", "dxs_dec")

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    SkyCoord(servs['servs_ra'], servs['servs_dec'])
)

In [ ]:
# Given the graph above, we use 1.2 arc-second radius
masterlist_catalogue = masterlist.merge_catalogues(
    masterlist_catalogue, servs, "servs_ra", "servs_dec")

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    SkyCoord(swire['swire_ra'], swire['swire_dec'])
)

In [ ]:
masterlist_catalogue = masterlist.merge_catalogues(
    masterlist_catalogue, swire, "swire_ra", "swire_dec")

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    SkyCoord(sparcs['sparcs_ra'], sparcs['sparcs_dec'])
)

In [ ]:
# Given the graph above, we use 1 arc-second radius
masterlist_catalogue = masterlist.merge_catalogues(
    masterlist_catalogue, sparcs, "sparcs_ra", "sparcs_dec", radius=1. * u.arcsec)

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    SkyCoord(panstarrs['panstarrs_ra'], panstarrs['panstarrs_dec'])
)

In [ ]:
# Given the graph above, we use ?????? arc-second radius
masterlist_catalogue = masterlist.merge_catalogues(
    masterlist_catalogue, panstarrs, "panstarrs_ra", "panstarrs_dec", radius=1. * u.arcsec)

In [ ]:
# When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes
# only from a catalogue and has no counterparts in the other, the columns from the latest
# are masked for that row). We indicate to use NaN for masked values for floats columns and
# False for flag columns.
for col in masterlist_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        masterlist_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        masterlist_catalogue[col].fill_value = False

In [ ]:
masterlist_catalogue[:10].show_in_notebook()

## VI - Merging flags and stellarity


In [ ]:
masterlist_catalogue.add_column(Column(
    data=(masterlist_catalogue['wfc_flag_cleaned'] | 
          masterlist_catalogue['dxs_flag_cleaned'] |
          masterlist_catalogue['servs_flag_cleaned'] | 
          masterlist_catalogue['swire_flag_cleaned'] |
          masterlist_catalogue['sparcs_flag_cleaned']),
    name="flag_cleaned"
))
masterlist_catalogue.remove_columns(['wfc_flag_cleaned', 'dxs_flag_cleaned',
                                     'servs_flag_cleaned', 'swire_flag_cleaned',
                                     'sparcs_flag_cleaned'])

In [ ]:
masterlist_catalogue.add_column(Column(
    data=(masterlist_catalogue['wfc_flag_gaia'] | 
          masterlist_catalogue['dxs_flag_gaia'] |
          masterlist_catalogue['servs_flag_gaia'] | 
          masterlist_catalogue['swire_flag_gaia'] |
          masterlist_catalogue['sparcs_flag_gaia']),
    name="flag_gaia"
))
masterlist_catalogue.remove_columns(['wfc_flag_gaia', 'dxs_flag_gaia',
                                     'servs_flag_gaia', 'swire_flag_gaia',
                                     'sparcs_flag_gaia'])

In [ ]:
masterlist_catalogue.add_column(Column(
    data=np.nanmax([masterlist_catalogue['wfc_stellarity'],
                     masterlist_catalogue['dxs_stellarity'],
                     masterlist_catalogue['servs_stellarity_irac1'],
                     masterlist_catalogue['servs_stellarity_irac2'],
                     masterlist_catalogue['swire_stellarity_irac1'],
                     masterlist_catalogue['swire_stellarity_irac2'],
                     masterlist_catalogue['swire_stellarity_irac3'],
                     masterlist_catalogue['swire_stellarity_irac4'],
                     masterlist_catalogue['sparcs_stellarity']],
                   axis=0),
    name='stellarity'
))
masterlist_catalogue.remove_columns(
    ['wfc_stellarity',
     'dxs_stellarity',
     'servs_stellarity_irac1',
     'servs_stellarity_irac2',
     'swire_stellarity_irac1',
     'swire_stellarity_irac2',
     'swire_stellarity_irac3',
     'swire_stellarity_irac4',
     'sparcs_stellarity']
)

## VII - E(B-V)

In [ ]:
masterlist_catalogue.add_column(
    utils.ebv(masterlist_catalogue['ra'], masterlist_catalogue['dec'])
)

## VIII - Choosing between multiple values for the same filter

We have IRAC1 and IRAC2 observation for both SERVS and SWIRE.  We must choose which one to use, remove the unused columns, and rename the ones we keep to names like `f_irac1`...

# IX - Domains of observation

We define three wavelength domains: optical, near-infrared, and mid-infrated (IRAC). We add binary flags combining in an integer the three domains:

- 1 for observation if optical;
- 2 for observation in near-infrared;
- 4 for observation in mid-infrared (IRAC).

We add two flags:

- `flag_optnir_obs` indicating if the source was observed in this wavelength domain (i.e. is on the coverage of the catalogue;
- `flag_optnir_flux` indicating if the source has measured flux in this wavelengh domain.

*Note 1: We use the total flux columns to know if the source has flux, in some catalogues, we may have aperture flux and no total flux.*

*Note 2: The observation flag is based on the creation of multi-order coverage maps from the catalogues, this may not be accurate, especially on the edges of the coverage.*

*Note 3: For sources observed in one domain but having no flux in it, one must take into consideration de different depths in the catalogue we are using.*

In [ ]:
was_observed_optical = utils.inMoc(
    masterlist_catalogue['ra'], masterlist_catalogue['dec'],
    wfc_moc + sparcs_moc + panstarrs_moc) 

was_observed_nir = utils.inMoc(
    masterlist_catalogue['ra'], masterlist_catalogue['dec'],
    dxs_moc
)

was_observed_mir = utils.inMoc(
    masterlist_catalogue['ra'], masterlist_catalogue['dec'],
    servs_moc + swire_moc
)

In [ ]:
masterlist_catalogue.add_column(
    Column(
        1 * was_observed_optical + 2 * was_observed_nir + 4 * was_observed_mir,
        name="flag_optnir_obs")
)

In [ ]:
has_optical_flux = (
    ~np.isnan(masterlist_catalogue['f_wfc_u'].filled()) |
    ~np.isnan(masterlist_catalogue['f_wfc_g'].filled()) |
    ~np.isnan(masterlist_catalogue['f_wfc_r'].filled()) |
    ~np.isnan(masterlist_catalogue['f_wfc_i'].filled()) |
    ~np.isnan(masterlist_catalogue['f_wfc_z'].filled()) |
    ~np.isnan(masterlist_catalogue['f_cfht_megacam_u'].filled()) |
    ~np.isnan(masterlist_catalogue['f_cfht_megacam_g'].filled()) |
    ~np.isnan(masterlist_catalogue['f_cfht_megacam_r'].filled()) |
    ~np.isnan(masterlist_catalogue['f_cfht_megacam_z'].filled())
)

has_nir_flux = (
    ~np.isnan(masterlist_catalogue['f_ukidss_j'].filled()) |
    ~np.isnan(masterlist_catalogue['f_ukidss_h'].filled()) |
    ~np.isnan(masterlist_catalogue['f_ukidss_k'].filled())
)

has_mir_flux = (
    ~np.isnan(masterlist_catalogue['f_servs_irac1'].filled()) |
    ~np.isnan(masterlist_catalogue['f_servs_irac2'].filled()) |
    ~np.isnan(masterlist_catalogue['f_swire_irac1'].filled()) |
    ~np.isnan(masterlist_catalogue['f_swire_irac2'].filled()) |
    ~np.isnan(masterlist_catalogue['f_swire_irac3'].filled()) |
    ~np.isnan(masterlist_catalogue['f_swire_irac4'].filled())
)

In [ ]:
masterlist_catalogue.add_column(
    Column(
        1 * has_optical_flux + 2 * has_nir_flux + 4 * has_mir_flux,
        name="flag_optnir_flux")
)

In [ ]:
flag_obs = masterlist_catalogue['flag_optnir_obs']
flag_flux = masterlist_catalogue['flag_optnir_flux']

In [ ]:
venn3(
    [
        np.sum(flag_obs == 4),
        np.sum(flag_obs == 2),
        np.sum(flag_obs == 6),
        np.sum(flag_obs == 1),
        np.sum(flag_obs == 5),
        np.sum(flag_obs == 3),
        np.sum(flag_obs == 7)
    ],
    set_labels=('Optical', 'near-IR', 'mid-IR'),
    subset_label_formatter=lambda x: "{}%".format(int(100*x/len(flag_obs)))
)
plt.title("Wavelength domain observations")

In [ ]:
venn3(
    [
        np.sum(flag_flux[flag_obs == 7] == 4),
        np.sum(flag_flux[flag_obs == 7] == 2),
        np.sum(flag_flux[flag_obs == 7] == 6),
        np.sum(flag_flux[flag_obs == 7] == 1),
        np.sum(flag_flux[flag_obs == 7] == 5),
        np.sum(flag_flux[flag_obs == 7] == 3),
        np.sum(flag_flux[flag_obs == 7] == 7)
    ],
    set_labels=('mid-IR', 'near-IR', 'Optical'),
    subset_label_formatter=lambda x: "{}%".format(int(100*x/np.sum(flag_flux != 0)))
)
plt.title("Flux availability for the {} sources observed\n in all wavelength domains "
          "(among {} sources)".format(
              locale.format('%d', np.sum(flag_flux != 0), grouping=True),
              locale.format('%d', len(flag_flux), grouping=True)))

## X - Number of bands with observation

In [ ]:
# We consider that a source is observed in a band when both the aperture and the full
# magnitudes are provided (some buggy catalogues miss some).

nb_bands = np.zeros(len(masterlist_catalogue), dtype=int)
for col in masterlist_catalogue.colnames:
    if 'm_app_' in col:
        fullmag_col = col.replace('m_app_', 'm_')
        nb_bands += (
            ~np.isnan(masterlist_catalogue[col].filled())
            & ~np.isnan(masterlist_catalogue[fullmag_col].filled())
        )

In [ ]:
sns.distplot(nb_bands, kde=False)

## X - Adding the HELP Id and field columns

In [ ]:
masterlist_catalogue.add_column(Column(
    utils.gen_help_id(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    name="help_id"
))
masterlist_catalogue.add_column(Column(
    np.full(len(masterlist_catalogue), "Lockman-SWIRE", dtype='<U18'),
    name="field"
))

In [ ]:
# Check that the HELP Ids are unique
if len(masterlist_catalogue) != len(np.unique(masterlist_catalogue['help_id'])):
    print("The HELP IDs are not unique!!!")
else:
    print("OK!")

## XI - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogue. This can be used to easily get additional information from them.

In [ ]:
cross_ident_table = masterlist_catalogue[
    'help_id', 'wfc_id', 'dxs_id', 'servs_intid', 'swire_intid', 'sparcs_intid', 'panstarrs_intid']
cross_ident_table.write("data/master_list_cross_ident_lockman-swire.fits")

## XII - Cleaning and saving the master catalogue

In [ ]:
masterlist_catalogue.remove_columns([
    'wfc_id', 'dxs_id', 'servs_intid', 'swire_intid', 'sparcs_intid', 'panstarrs_intid'])

In [ ]:
# We may want to reorder the column even if this will be done at the ingestion in HeDaM.
masterlist_catalogue.write("data/master_catalogue_elais-n1.fits")